In [2]:
import pandas as pd
from datetime import datetime as dt

import integrate as intr


In [3]:

miv_df = intr.process_miv_data()
fb_data = intr.process_foot_bike_data()

/Users/seb/Projects/repos/group-1/src/integrate.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miv_df_cols_dropped['Weekday_en'] = days.map(lambda x: weekday_names[x])


In [ ]:
test_df = miv_df


In [ ]:
test_df.dtypes
date_object = pd.to_datetime(test_df['Date'])


In [ ]:
test_df['Weekday_Name']

In [4]:
acc_df = intr.process_accident_data()

u_string RoadTrafficAccidentLocations.json
Filename: 2017_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: RoadTrafficAccidentLocations.json
Filepath: datasets/RoadTrafficAccidentLocations.json
Filename: 2016_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2022_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2015_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2019_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2013.csv
Filename: 2021_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2012.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2016.csv
Filename: 2014_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2018_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2017.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2015.csv
Filename: 2020_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_m

In [ ]:
acc_df.head()
acc_df['AccidentWeekDay'].unique()
#acc_df.dtypes
date_obj = dt.strptime(acc_df[''])


In [5]:
print("Accident Columns:")
print(acc_df.dtypes)
print()
print("MIV Columns:")
print(miv_df.dtypes)
print()
print("FB Cols:")
print(fb_data.dtypes)

Accident Columns:
AccidentUID                      object
AccidentHour                     object
AccidentYear                     object
AccidentWeekDay_en               object
AccidentType                     object
AccidentSeverityCategory         object
AccidentInvolvingPedestrian      object
AccidentInvolvingBicycle         object
AccidentInvolvingMotorcycle      object
RoadType                         object
RoadType_en                      object
AccidentLocation_CHLV95_E        object
AccidentLocation_CHLV95_N        object
geometry                       geometry
dtype: object

MIV Columns:
MSID                   object
ZSID                   object
Achse                  object
EKoord                float64
NKoord                float64
Richtung               object
AnzFahrzeuge          float64
AnzFahrzeugeStatus     object
Date                   object
Hrs                    object
Weekday_en             object
dtype: object

FB Cols:
OST             int64
NORD            in

In [13]:
print("MIV unqiue:", miv_df['EKoord'].unique().shape)
print("Acc unique:", acc_df['AccidentLocation_CHLV95_E'].unique().shape)
print("FB unique: ", fb_data['OST'].unique())


MIV unqiue: (187,)
Acc unique: (8661,)
FB unique:  (62,)


In [7]:
acc_df.head()

,AccidentUID,AccidentHour,AccidentYear,AccidentWeekDay_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,geometry
0,A2D2677533867004E0430A865E337004,00,2011,Saturday,at0,as4,false,false,false,rt433,Minor road,2684605,1245194,POINT Z (8.55841 47.35217 0.00000)
1,9FD6441F802C20A6E0430A865E3320A6,01,2011,Saturday,at0,as3,false,true,false,rt433,Minor road,2682382,1246980,POINT Z (8.52932 47.36851 0.00000)
2,9FDA0DC4856A6094E0430A865E336094,02,2011,Saturday,at0,as4,false,false,false,rt439,Other,2682791,1247749,POINT Z (8.53488 47.37538 0.00000)
3,A3B66E42396E6000E0430A865E336000,02,2011,Saturday,at5,as3,false,false,false,rt433,Minor road,2681199,1247102,POINT Z (8.51368 47.36976 0.00000)
4,9FDA0DBE8CCE9096E0430A865E339096,03,2011,Saturday,at0,as4,false,false,false,rt433,Minor road,2682479,1250690,POINT Z (8.53129 47.40186 0.00000)


In [17]:
from sqlalchemy import create_engine

db_url = f'postgresql://seb:@localhost:5432/test-db23'
engine =  create_engine(db_url)

#miv_df.to_sql('table_name', engine, if_exists='replace', index=False)
#fb_data.to_sql('footbike', engine, if_exists='replace', index=False)
acc_df.to_sql('table_name', engine, if_exists='replace', index=False)


ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'Point'
[SQL: INSERT INTO table_name ("AccidentUID", "AccidentHour", "AccidentYear", "AccidentWeekDay_en", "AccidentType", "AccidentSeverityCategory", "AccidentInvolvingPedestrian", "AccidentInvolvingBicycle", "AccidentInvolvingMotorcycle", "RoadType", "RoadType_e ... 398437 characters truncated ... n__999)s, %(AccidentLocation_CHLV95_E__999)s, %(AccidentLocation_CHLV95_N__999)s, %(geometry__999)s)]
[parameters: {'AccidentYear__0': '2011', 'AccidentLocation_CHLV95_N__0': '1245194', 'AccidentType__0': 'at0', 'AccidentSeverityCategory__0': 'as4', 'RoadType_en__0': 'Minor road', 'AccidentLocation_CHLV95_E__0': '2684605', 'AccidentWeekDay_en__0': 'Saturday', 'AccidentInvolvingMotorcycle__0': 'false', 'RoadType__0': 'rt433', 'AccidentUID__0': 'A2D2677533867004E0430A865E337004', 'AccidentInvolvingPedestrian__0': 'false', 'AccidentHour__0': '00', 'geometry__0': <POINT Z (8.558 47.352 0)>, 'AccidentInvolvingBicycle__0': 'false', 'A

In [16]:
engine.dispose()